# Creating the main_cell & JJ and JJ-Array

In [ ]:
import gdspy
from importlib import reload
from itertools import product
reload(gdspy)
lib = gdspy.GdsLibrary()


#######################################################################################

#create base layer with main ground and markers
main_cell = lib.new_cell('main_cell')

unground = gdspy.Rectangle((-4750,-4750), (4750, 4750), layer=2)
main_cell.add(unground)

#create markers
id_ = [(-1, 1), (1, 1), (-1, -1), (1, -1)]

length_sq = 1000

for i in id_:
    corner = (i[0]*4750 - i[0]*250, i[1]*4750 - i[1]*250)
    center = (i[0]*4750 + i[0]*250, i[1]*4750 + i[1]*250)
    print(center)
    
    gr =  gdspy.Rectangle((corner[0] + i[0]*length_sq, corner[1]), (corner[0], corner[1] + i[1]*length_sq), layer=2)
    main_cell.add(gr)
    
    for j in id_:
        w = 20
        marker = gdspy.Rectangle((center[0] + j[0]*100 - w, center[1]+ j[1]*100 - w),
                                   (center[0] + j[0]*100 + w, center[1]+ j[1]*100 + w),  layer=3)
        
        main_cell.add(marker)
        
        
#logo
logo = gdspy.Text("Andersen Lab/QuTech", 400, (-2400, -5400), layer=2)
main_cell.add(logo)


# adding 4 square on the main cell for deposition, there is no arrow. I will fix this in the future.

arrow1_part1 = gdspy.Rectangle((-3400+700, 5300 - 500), (-3400+700+3700, 5300 + 500), layer=100)
main_cell.add(arrow1_part1)

arrow1_part2 = gdspy.Rectangle((-3400+700+3700+1100, 5300 - 500), (-3400+700+3700+1100+1000, 5300 + 500), layer=100)
main_cell.add(arrow1_part2)


arrow2_part1 = gdspy.Rectangle((5300 - 500, -2000+700), (5300 + 500, 2400), layer=100)
main_cell.add(arrow2_part1)

arrow2_part2 = gdspy.Rectangle((5300 - 500, -3400), (5300 + 500, -2400), layer=100)
main_cell.add(arrow2_part2)


#######################################################################################

##### N SHOULD BE EVEN!!!!!!!!!!
# N is a number of JJ in the chain
N = 100

if int(N / 2) % 2 != 0:
    raise ValueError('N / 2 should be even!!!')

delta_x = 0.5
# delta_x_base = 1.5
delta_x_base = 1.5
# nanowire_length is  the length between nanowire's pads
nanowire_length = 130 + 3
real_nanowire_length = nanowire_length - delta_x * 2 - delta_x_base * 2

# very important shift at x direction
SHIFT = - nanowire_length / 2
nanowire_width = 1.3

delta = 0.460  # initial width of pad

nu_cell = 0

start_x = 0
start_y = 0

stop_x = start_x + nanowire_length
stop_y = start_y

# for vertical and horizontallines
step = 1.2
delta_W = (real_nanowire_length - 2 * step) / (N / 2 - 1)
delta_H = 3
print(delta_W)

corr1 = 1.9
corr2 = 1

u = 1

if delta == 0.460:
    cell = lib.new_cell('jj_chain' + str(nu_cell))
    
# Bandage for the left    
    ban1 = gdspy.Rectangle((start_x + delta_x_base - 1.5 + SHIFT - 0.2, start_y - 0.7 / 2 - 0.2),
                           (start_x + delta_x_base + delta_x + SHIFT + 0.2, start_y + 0.7 / 2 + 0.2),
                          layer=56)
    
    cell.add(ban1)
    
    # left line
    
    left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x+SHIFT-1.6, start_y - delta / 2),
                                (start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT, start_y + delta / 2), layer=54)
    cell.add(left_line)
    
    undercut_left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT, start_y - delta / 2),
                                         (start_x + delta_x_base + delta_x + step + delta+corr2+0.7+SHIFT, start_y + delta / 2),
                                         layer=55)
    cell.add(undercut_left_line)
    
# Bandage for the right    
    ban2 = gdspy.Rectangle((stop_x + SHIFT -1.8, stop_y - 3.7),
                           (stop_x + SHIFT -1.8+2.6, stop_y - 3.7 + 1.1), 
                          layer=56)
    cell.add(ban2)
    
 
    chain_JJ = None
    k = 0 #  center
    p = 0 #  down 
    for i in range(int(N / 2)):
        
        if i<N//2-1:
            vertical_pad = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y - 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6),
                layer=54)


            undercut_vertical = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y + 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6 + u),

                layer=55)

            chain_JJ = gdspy.boolean(chain_JJ, vertical_pad, 'or')

            cell.add(vertical_pad)
            cell.add(undercut_vertical)
        if i==N//2-2:
            corr2 = 2.5
        if i % 2 != 0 and i<N//2-1: #and i != int(N / 2) - 1:
            
            if k % 2 == 0:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+ 3),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+ 3),
                layer=55)
                
            else:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+1),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2 +1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+1),
                layer=55)
                        
            k +=1
        
            chain_JJ = gdspy.boolean(chain_JJ, horizonatal_center, 'or')

            cell.add(horizonatal_center)

            cell.add(undercut_horizontal_center)

        if i % 2 == 0 and i<N//2-1:
            
            if p % 2 == 0:
                
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta),
                layer=55)
                
            else:
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta + 1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H + 1.5),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H+1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta+1.5),
                layer=55)
                
            p += 1

        
            chain_JJ = gdspy.boolean(chain_JJ, horizonatal_down, 'or')

            cell.add(horizonatal_down)

            cell.add(undercut_horizontal_center)
            
########################### ADDING JJ ###########################################################
            
deltaJJ = 0.100 # initial width of pad
pad_length_left = 2.4
pad_length_right = 1.7

nu_cellJJ = 0

if deltaJJ == 0.100:
            cell = lib.new_cell('junction_' + str(nu_cellJJ))
    
            left_big_points = [(-15., 0), (-10, 0), (-2., 1.5), (-2., 3), (-15, 3)]
            right_big_points = [(2., -1.5), (15, -1.5), (15., 1.5), (10, 1.5), (2., 0)]
    
            left_big = gdspy.Polygon(left_big_points, layer=53)
            right_big = gdspy.Polygon(right_big_points, layer=53)
    
            left_small = gdspy.Rectangle((-2., 1.5), (-0.2, 3), layer=53)
    
            right_small = gdspy.Rectangle((0.2, -1.5), (2., 0), layer=53)
    

        # Pads of junctions
            left_padJJ = gdspy.Rectangle((-1.2, -2.8), (-1.2 + deltaJJ, 2.3), layer=54)
    
            right_padJJ = gdspy.Rectangle((-3.0, -1.0), (1.2, -1.0 + deltaJJ), layer=54)
    
    
        # parts for undercuts
            undercut_left_pad3 = gdspy.Rectangle((-1.2, 2.3), (-1.2 + deltaJJ, + 3), layer=55)
    
            undercut_right_pad4 = gdspy.Rectangle((1.2, -1.0), (1.9, -1.0 + deltaJJ), layer=55)
    
        #bandages
            ban3 = gdspy.Rectangle((-1.7, 1.0), (-0.6, 3), layer=56)
            ban4 = gdspy.Rectangle((-0.25, -1.5), (2., -0.4), layer=56)
    

            cell.add(left_big)
            cell.add(right_big)

            cell.add(left_small)
            cell.add(right_small)

            cell.add(left_padJJ)
            cell.add(right_padJJ)
    
            cell.add(undercut_left_pad3)
            cell.add(undercut_right_pad4)
    
            cell.add(ban3)
            cell.add(ban4)
            



In [ ]:
gdspy.LayoutViewer()

In [ ]:
lib.write_gds('Fluxonium_JJandArray.gds')

# Let's try the GDS file that we just created. We can only use Q1 and see. 
# Create geometry of a chip


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
import matplotlib.pyplot as plt

from qiskit_metal.toolbox_metal import math_and_overrides

from qiskit_metal.qlibrary.core import QComponent

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

from itertools import product

In [ ]:
# Each time you create a new quantum circuit design,
# you start by instantiating a QDesign class.

# The design class `DesignPlanar` is best for 2D circuit designs.
design = designs.DesignPlanar()
design.chips.main.material = 'silicon'
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '8.733 um'
design._chips['main']['size']['size_x'] = '9mm'
design._chips['main']['size']['size_y'] = '9mm'

# If you disable the next line with "overwrite_enabled", then you will need to
# delete a component [<component>.delete()] before recreating it.
design.overwrite_enabled = True

gui = MetalGUI(design)

In [ ]:
from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

gds_cell_name_jj = 'junction_0'
gds_cell_name_inductor = 'jj_chain0'

# Q1 details
options = dict(chip='main', nanowire = False, 
            orientation =90, pos_x = '-2.2mm', pos_y = '-2.3mm',
            gds_cell_name=gds_cell_name_jj,
            gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
                        fbl_sep='150um',), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = True,
            cl_sep ='-10um'),
    readout_line_options=Dict(loc_H = -1, make_rol = True, 
            pad_width = '300um', pad_height = '80um',
            pad_sep='60um', 
              ) )
               
q1 = FluxoniumPocket(design,'Q1', options = dict(
             **options))

gui.rebuild()

In [ ]:
gui.main_window.close()

# Creating the GDS file and yes we are the GOD!

In [ ]:
fluxonium_gds = design.renderers.gds
fluxonium_gds.options

In [ ]:
# Choose file (with gradual junctions or note)
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions.gds'
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions_gradual.gds'
fluxonium_gds.options['path_filename'] = 'Fluxonium_JJandArray.gds'

In [ ]:
# fluxonium_gds.export_to_gds('fluxonium_calibration_structure.gds')
fluxonium_gds.export_to_gds('FluxoniumPocket-Q1-460nmArray100umJJ.gds')

In [ ]:
fluxonium_gds.options['fabricate'] = True